In [1]:
import sympy as sp

# Convert Between Formats
Using the shunting yard algo: https://brilliant.org/wiki/shunting-yard-algorithm/

### Tokenize equation variables

In [2]:
# Token class
class Token:
    def __init__(self, t_type, t_value=None):
        self.t_type = t_type
        self.t_value = t_value
        
    def __repr__(self):
        return f"---- Type: {self.t_type} \t Value: {self.t_value} ----"

In [3]:
# Tokens
## Variable
TT_VARIABLE = "TT_VARIABLE"

## Specials
TT_LEFT_PARENTHESIS = "TT_LEFT_PARENTHESIS"
TT_RIGHT_PARENTHESIS = "TT_RIGHT_PARENTHESIS"

## Special numbers
TT_PI = "TT_PI"
TT_E = "TT_E"
TT_PHI = "TT_PHI"
TT_CATALAN = "TT_CATALAN"

## Numbers
TT_INTEGER = "TT_INTEGER"
TT_RATIONAL = "TT_RATIONAL"

## Unary Operations
TT_SQRT = "TT_SQRT"
TT_SIN = "TT_SIN"
TT_COS = "TT_COS"
TT_TAN = "TT_TAN"

TT_FACTORIAL = "TT_FACTORIAL"

## Binary Operations
TT_PLUS = "TT_PLUS"
TT_MINUS = "TT_MINUS"
TT_MULTIPLY = "TT_MULTIPLY"
TT_DIVIDE = "TT_DIVIDE"

In [4]:
# Helper globals
DIGITS = "0123456789"
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVXYZabcdefghijklmnopqrstuvxyz"

SPECIAL_NUMBERS_DICT = {
    "pi": Token(TT_PI),
    "e": Token(TT_E),
    "phi": Token(TT_PHI),
    "G": Token(TT_CATALAN)
}

UNI_OPERATORS = {
    "sqrt": Token(TT_SQRT),
    "sin": Token(TT_SIN),
    "cos": Token(TT_COS),
    "tan": Token(TT_TAN)
}
BIN_OPERATORS = {
    TT_PLUS: "+",
    TT_MINUS: "-",
    TT_MULTIPLY: "*",
    TT_DIVIDE: "/"
}
NUMBERS = {
    TT_PI: "pi",
    TT_E: "e",
    TT_PHI: "phi",
    TT_CATALAN: "G",
    TT_INTEGER: ""
}

### Lexer implementation

In [7]:
class EquationLexer:
    def __init__(self, text):
        self.text = text
        self.position = -1
        self.text_length = len(text)
        self.current_char = None
        self.advance()
    
    ###########################
    # Convert to tokens logic #
    ###########################
    
    # Advances to next character
    def advance(self):
        self.position += 1
        self.current_char = self.text[self.position] if (self.position < self.text_length) else None
    
    def make_tokens(self):
        tokens = []
        
        while (self.current_char != None):
            if self.current_char == " ":
                self.advance()
            elif self.current_char in DIGITS:
                tokens.append(self._makeInteger())
            elif self.current_char in ALPHABET:
                tokens.append(self._makeFunc())
            elif self.current_char == "/":
                tokens.append(Token(TT_DIVIDE))
                self.advance()
            elif self.current_char == "+":
                tokens.append(Token(TT_PLUS))
                self.advance()
            elif self.current_char == "-":
                tokens.append(Token(TT_MINUS))
                self.advance()
            elif self.current_char == "*":
                tokens.append(Token(TT_MULTIPLY))
                self.advance()
            elif self.current_char == "(":
                tokens.append(Token(TT_LEFT_PARENTHESIS))
                self.advance()
            elif self.current_char == ")":
                tokens.append(Token(TT_RIGHT_PARENTHESIS))
                self.advance()
            elif self.current_char == "!":
                tokens.append(Token(TT_FACTORIAL))
                self.advance()
            elif self.current_char in ALPHABET:
                tokens.append(self._makeFunc())
            else:
                print(f"ERROR unrecognised token: {self.current_char}")
                return []
        
        return tokens
    
    def _makeInteger(self):
        res = ""
        
        while self.current_char in DIGITS:
            res += self.current_char
            self.advance()
        
        return Token(TT_INTEGER, int(res))
    
    def _makeFunc(self):
        res = ""
        
        while self.current_char in ALPHABET:
            res += self.current_char
            self.advance()
        
        # Map to token type and return it
        ## If: pi,e,phi,...
        if res in SPECIAL_NUMBERS_DICT:
            return SPECIAL_NUMBERS_DICT[res]
        
        ## If: x,y,z,m,n,...
        elif len(res) == 1:
            return Token(TT_VARIABLE, res)
        
        ## If: sqrt,sin,cos,...
        elif res in UNI_OPERATORS:
            return UNI_OPERATORS[res]

In [8]:
equation = EquationLexer("3+18/(9-3)+sin(pi)")
equation.make_tokens()

[---- Type: TT_INTEGER 	 Value: 3 ----,
 ---- Type: TT_PLUS 	 Value: None ----,
 ---- Type: TT_INTEGER 	 Value: 18 ----,
 ---- Type: TT_DIVIDE 	 Value: None ----,
 ---- Type: TT_LEFT_PARENTHESIS 	 Value: None ----,
 ---- Type: TT_INTEGER 	 Value: 9 ----,
 ---- Type: TT_MINUS 	 Value: None ----,
 ---- Type: TT_INTEGER 	 Value: 3 ----,
 ---- Type: TT_RIGHT_PARENTHESIS 	 Value: None ----,
 ---- Type: TT_PLUS 	 Value: None ----,
 ---- Type: TT_SIN 	 Value: None ----,
 ---- Type: TT_LEFT_PARENTHESIS 	 Value: None ----,
 ---- Type: TT_PI 	 Value: None ----,
 ---- Type: TT_RIGHT_PARENTHESIS 	 Value: None ----]

### Notation converter implementation

In [9]:
class Equation:
    def __init__(self, tokenized_equation:list=None, notation:str="infix"):
        self.tokenized_equation = tokenized_equation
        self.notation = notation
        
    def convertToInfix(self):
        pass
    
    def convertToPostfix(self):
        if self.notation == "infix":
            token_list = self.tokenized_equation
            operator_stack = []
            output_queue = []
            #################
            # Shunting yard #
            #################
            for token in token_list:
                if token.t_type in UNI_OPERATORS:
                    operator_stack.append(token)
                elif token.t_type in BIN_OPERATORS:
                    pass
                
                
    def convertToPrefix(self):
        pass
    
    def _operatorPrecedenceComparison(self, operator_1: Token, operator_2: Token):
        """Compares operators in input for higher precedence
        
        Args:
            operator_1 (Token)
            operator_2 (Token)
            
        Returns:
            0 if operator_1 has higher precedence than operator_2
            1 if operator_1 has equal precedence as operator_2
            2 if operator_1 has less precedence than operator_2
        """
        return
    
    @classmethod
    def makeEquationFromString(cls, equation:str, notation="infix"):
        """Makes equation instance from string
        
        Args:
            equation (str): the equation to use in string format
            notation (str): the notation used in the string equation
        
        Returns:
            Instance of the Equation class initialized from equation
        """
        lexer = EquationLexer(equation)
        tokenized_equation = lexer.make_tokens()
        return cls(tokenized_equation, notation)